In [1]:
import pandas as pd
import heapq
import pickle
import numpy as np
import seaborn as sns
import random
import matplotlib.patches as mpatches
import matplotlib as mpl
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import time
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn import svm, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from tabulate import tabulate
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.linear_model import Perceptron
from timeit import timeit
from sklearn import datasets, tree
import datetime
import os
import warnings
warnings.simplefilter(action = 'ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
import pylab 
sns.set(style="ticks", color_codes=True, font_scale=1.5)
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
%matplotlib inline
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
#from graphviz import Source
from IPython.display import Image
from scipy.stats import skew, norm, probplot, boxcox, f_oneway
from scipy import interp
from sklearn.base import BaseEstimator, TransformerMixin, clone, ClassifierMixin
from sklearn import metrics, tree
from sklearn.preprocessing import LabelEncoder, label_binarize, StandardScaler, PolynomialFeatures, MinMaxScaler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, cross_val_predict, train_test_split, RandomizedSearchCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from sklearn.linear_model import LogisticRegression
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.tree import DecisionTreeClassifier
#import xgboost as xgb
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.cluster import DBSCAN
import scipy
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from google.cloud import bigquery
from sklearn.model_selection import TimeSeriesSplit
from scipy.stats import chi2_contingency
import joblib

In [2]:
# set display options to show all columns
pd.set_option('display.max_columns', None)
# Set the float format to display numbers without scientific notation
pd.options.display.float_format = '{:.2f}'.format
# Set the client for future queries to BigQuery
client = bigquery.Client(project = "continente-lced-feup")

In [3]:
query = client.query("""
   SELECT *
   FROM 
       tables_staging.df_bold_model
   ORDER BY CUSTOMER_ACCOUNT_NR_MASK ASC, SUBCAT_CD_EXT ASC, YEAR ASC, MONTH ASC
   """)

df = query.result().to_dataframe() # Wait for the job to complete.
df # Selected the top1000 customers with the most number of transactions in the last 360 days in 12/2021

,CUSTOMER_ACCOUNT_NR_MASK,SUBCAT_CD_EXT,MONTH,YEAR,QUARTER,SEMESTER,CUST_NUM_TRANSACTIONS_MONTH,CUST_NUM_TRANSACTIONS_QUARTER,CUST_NUM_TRANSACTIONS_SEMESTER,CUST_NUM_TRANSACTIONS_YEAR,CUST_TOTAL_QTY_BOUGHT_MONTH,CUST_TOTAL_QTY_BOUGHT_QUARTER,CUST_TOTAL_QTY_BOUGHT_SEMESTER,CUST_TOTAL_QTY_BOUGHT_YEAR,CUST_NUM_UNIQUE_SUBCAT_MONTH,CUST_NUM_UNIQUE_SUBCAT_QUARTER,CUST_NUM_UNIQUE_SUBCAT_SEMESTER,CUST_NUM_UNIQUE_SUBCAT_YEAR,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR,REG_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH,CUST_AVG_BASKET_SIZE_MONTH,CUST_AVG_BASKET_SIZE_QUARTER,CUST_AVG_BASKET_SIZE_SEMESTER,CUST_AVG_BASKET_SIZE_YEAR,SUBCAT_NUM_TRANSACTIONS_MONTH,SUBCAT_NUM_TRANSACTIONS_QUARTER,SUBCAT_NUM_TRANSACTIONS_SEMESTER,SUBCAT_NUM_TRANSACTIONS_YEAR,SUBCAT_TOTAL_QTY_BOUGHT_MONTH,SUBCAT_TOTAL_QTY_BOUGHT_QUARTER,SUBCAT_TOTAL_QTY_BOUGHT_SEMESTER,SUBCAT_TOTAL_QTY_BOUGHT_YEAR,SUBCAT_NUM_UNIQUE_CUST_MONTH,SUBCAT_NUM_UNIQUE_CUST_QUARTER,SUBCAT_NUM_UNIQUE_CUST_SEMESTER,SUBCAT_NUM_UNIQUE_CUST_YEAR,CUSTSUBCAT_NUM_TRANSACTIONS_MONTH,CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER,CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER,CUSTSUBCAT_NUM_TRANSACTIONS_YEAR,CUSTSUBCAT_TOTAL_QTY_BOUGHT_MONTH,CUSTSUBCAT_TOTAL_QTY_BOUGHT_QUARTER,CUSTSUBCAT_TOTAL_QTY_BOUGHT_SEMESTER,CUSTSUBCAT_TOTAL_QTY_BOUGHT_YEAR,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR,GENDER,FAMILY_MEMBERS,SEG_LIFESTYLE_CD,SEG_LIFESTAGE_CD,CAT_CD_EXT,TARGET
0,37912,10101,1,2021,1,1,5,5,5,5,24,24,24,24,12,12,12,12,2.25,2.25,2.25,2.25,4.20,3.60,3.60,3.60,3.60,2733,2733,2733,2733,4304,4304,4304,4304,1741,1741,1741,1741,0,0,0,0,0,0,0,0,0.00,0,0,0,M,"(3, 8)",2,4,101,0
1,37912,10101,2,2021,1,1,6,11,11,11,46,70,70,70,20,27,27,27,4.20,5.30,5.30,5.30,2.40,5.50,4.64,4.64,4.64,2640,5373,5373,5373,4140,8444,8444,8444,1676,2504,2504,2504,0,0,0,0,0,0,0,0,0.00,0,0,0,M,"(3, 8)",2,4,101,1
2,37912,10101,3,2021,1,1,11,22,22,22,60,130,130,130,21,37,37,37,2.40,3.86,3.86,3.86,5.20,3.55,4.09,4.09,4.09,2924,8297,8297,8297,4686,13130,13130,13130,1863,3090,3090,3090,2,2,2,2,3,3,3,3,9.00,9,9,9,M,"(3, 8)",2,4,101,1
3,37912,10101,4,2021,2,1,6,6,28,28,29,29,159,159,13,13,39,39,5.20,5.20,4.11,4.11,6.00,4.17,4.17,4.11,4.11,2577,2577,10874,10874,4200,4200,17330,17330,1760,1760,3511,3511,2,2,4,4,4,4,7,7,12.00,12,13,13,M,"(3, 8)",2,4,101,1
4,37912,10101,5,2021,2,1,5,11,33,33,31,60,190,190,13,20,44,44,6.00,5.50,4.38,4.38,3.57,4.20,4.18,4.12,4.12,2824,5401,13698,13698,4815,9015,22145,22145,1885,2731,3890,3890,1,3,5,5,1,5,8,8,0.00,20,17,17,M,"(3, 8)",2,4,101,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3365383,130366108,170313,8,2022,3,2,9,21,21,55,67,142,142,463,25,43,43,67,3.00,2.70,2.70,4.28,2.67,6.11,5.19,5.19,5.84,106,191,191,618,190,318,318,912,93,158,158,443,0,1,1,2,0,1,1,2,0.00,0,0,12,M,"(3, 8)",1,4,1703,0
3365384,130366108,170313,9,2022,3,2,10,31,31,65,76,218,218,539,25,51,51,69,2.67,2.93,2.93,4.14,4.00,5.80,5.39,5.39,5.83,67,258,258,685,89,407,407,1001,63,207,207,485,0,1,1,2,0,1,1,2,0.00,0,0,12,M,"(3, 8)",1,4,1703,0
3365385,130366108,170313,10,2022,4,2,7,7,38,72,18,18,236,557,10,10,54,70,4.00,4.00,3.27,4.20,4.80,1.86,1.86,4.74,5.44,78,78,336,763,118,118,525,1119,71,71,268,533,0,0,1,2,0,0,1,2,0.00,0,0,12,M,"(3, 8)",1,4,1703,0
3365386,130366108,170313,11,2022,4,2,6,13,44,78,54,72,290,611,18,24,57,71,4.80,4.33,3.47,4.23,2.75,4.33,3.00,4.68,5.36,77,155,413,840,105,223,630,1224,65,131,316,568,0,0,1,2,0,0,1,2,0.00,0,0,12,M,"(3, 8)",1,4,1703,0


In [4]:
df_pipeline = df.copy()

# Data Preparation

In [5]:
df_pipeline['FULLDATE'] = pd.to_datetime(df_pipeline['MONTH'].astype(str) + '-' + df_pipeline['YEAR'].astype(str))
# change from dtype datetime64[ns] to date time month
df_pipeline['FULLDATE'] = df_pipeline['FULLDATE'].dt.to_period('M')

df_pipeline.drop(columns=['CUSTOMER_ACCOUNT_NR_MASK','REG_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH'], inplace=True)

In [6]:
# Function to replace outliers with average value of the column
def replace_outliers_with_average(df_non_minmax, column):
    # Calculate the lower and upper bounds for outliers using the boxplot rule
    q1 = df_non_minmax[column].quantile(0.25)
    q3 = df_non_minmax[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Replace outliers with the average value of the column
    avg_value = df_non_minmax[column].mean()
    df_non_minmax[column] = df_non_minmax[column].apply(lambda x: avg_value if x < lower_bound or x > upper_bound else x)
    
    return df_non_minmax

# Detect outliers and replace them with average for each column in 'columns_to_check'
numerical_columns = ['CUST_NUM_TRANSACTIONS_MONTH', 'CUST_NUM_TRANSACTIONS_QUARTER',
       'CUST_NUM_TRANSACTIONS_SEMESTER', 'CUST_NUM_TRANSACTIONS_YEAR',
       'CUST_TOTAL_QTY_BOUGHT_MONTH', 'CUST_TOTAL_QTY_BOUGHT_QUARTER',
       'CUST_TOTAL_QTY_BOUGHT_SEMESTER', 'CUST_TOTAL_QTY_BOUGHT_YEAR',
       'CUST_NUM_UNIQUE_SUBCAT_MONTH', 'CUST_NUM_UNIQUE_SUBCAT_QUARTER',
       'CUST_NUM_UNIQUE_SUBCAT_SEMESTER', 'CUST_NUM_UNIQUE_SUBCAT_YEAR',
       'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH',
       'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER',
       'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER',
       'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR',
       'CUST_AVG_BASKET_SIZE_MONTH', 'CUST_AVG_BASKET_SIZE_QUARTER',
       'CUST_AVG_BASKET_SIZE_SEMESTER', 'CUST_AVG_BASKET_SIZE_YEAR',
       'SUBCAT_NUM_TRANSACTIONS_MONTH', 'SUBCAT_NUM_TRANSACTIONS_QUARTER',
       'SUBCAT_NUM_TRANSACTIONS_SEMESTER', 'SUBCAT_NUM_TRANSACTIONS_YEAR',
       'SUBCAT_TOTAL_QTY_BOUGHT_MONTH', 'SUBCAT_TOTAL_QTY_BOUGHT_QUARTER',
       'SUBCAT_TOTAL_QTY_BOUGHT_SEMESTER', 'SUBCAT_TOTAL_QTY_BOUGHT_YEAR',
       'SUBCAT_NUM_UNIQUE_CUST_MONTH', 'SUBCAT_NUM_UNIQUE_CUST_QUARTER',
       'SUBCAT_NUM_UNIQUE_CUST_SEMESTER', 'SUBCAT_NUM_UNIQUE_CUST_YEAR',
       'CUSTSUBCAT_NUM_TRANSACTIONS_MONTH',
       'CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER',
       'CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER',
       'CUSTSUBCAT_NUM_TRANSACTIONS_YEAR', 'CUSTSUBCAT_TOTAL_QTY_BOUGHT_MONTH',
       'CUSTSUBCAT_TOTAL_QTY_BOUGHT_QUARTER',
       'CUSTSUBCAT_TOTAL_QTY_BOUGHT_SEMESTER',
       'CUSTSUBCAT_TOTAL_QTY_BOUGHT_YEAR',
       'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH',
       'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER',
       'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER',
       'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR']

for column in numerical_columns:
    df_pipeline = replace_outliers_with_average(df_pipeline, column)

In [7]:
scaler = MinMaxScaler()
df_pipeline[numerical_columns] = scaler.fit_transform(df_pipeline[numerical_columns])
df_pipeline.head(20)

,SUBCAT_CD_EXT,MONTH,YEAR,QUARTER,SEMESTER,CUST_NUM_TRANSACTIONS_MONTH,CUST_NUM_TRANSACTIONS_QUARTER,CUST_NUM_TRANSACTIONS_SEMESTER,CUST_NUM_TRANSACTIONS_YEAR,CUST_TOTAL_QTY_BOUGHT_MONTH,CUST_TOTAL_QTY_BOUGHT_QUARTER,CUST_TOTAL_QTY_BOUGHT_SEMESTER,CUST_TOTAL_QTY_BOUGHT_YEAR,CUST_NUM_UNIQUE_SUBCAT_MONTH,CUST_NUM_UNIQUE_SUBCAT_QUARTER,CUST_NUM_UNIQUE_SUBCAT_SEMESTER,CUST_NUM_UNIQUE_SUBCAT_YEAR,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER,CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR,CUST_AVG_BASKET_SIZE_MONTH,CUST_AVG_BASKET_SIZE_QUARTER,CUST_AVG_BASKET_SIZE_SEMESTER,CUST_AVG_BASKET_SIZE_YEAR,SUBCAT_NUM_TRANSACTIONS_MONTH,SUBCAT_NUM_TRANSACTIONS_QUARTER,SUBCAT_NUM_TRANSACTIONS_SEMESTER,SUBCAT_NUM_TRANSACTIONS_YEAR,SUBCAT_TOTAL_QTY_BOUGHT_MONTH,SUBCAT_TOTAL_QTY_BOUGHT_QUARTER,SUBCAT_TOTAL_QTY_BOUGHT_SEMESTER,SUBCAT_TOTAL_QTY_BOUGHT_YEAR,SUBCAT_NUM_UNIQUE_CUST_MONTH,SUBCAT_NUM_UNIQUE_CUST_QUARTER,SUBCAT_NUM_UNIQUE_CUST_SEMESTER,SUBCAT_NUM_UNIQUE_CUST_YEAR,CUSTSUBCAT_NUM_TRANSACTIONS_MONTH,CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER,CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER,CUSTSUBCAT_NUM_TRANSACTIONS_YEAR,CUSTSUBCAT_TOTAL_QTY_BOUGHT_MONTH,CUSTSUBCAT_TOTAL_QTY_BOUGHT_QUARTER,CUSTSUBCAT_TOTAL_QTY_BOUGHT_SEMESTER,CUSTSUBCAT_TOTAL_QTY_BOUGHT_YEAR,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_MONTH,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_QUARTER,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_SEMESTER,CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION_YEAR,GENDER,FAMILY_MEMBERS,SEG_LIFESTYLE_CD,SEG_LIFESTAGE_CD,CAT_CD_EXT,TARGET,FULLDATE
0,10101,1,2021,1,1,0.45,0.18,0.09,0.05,0.15,0.08,0.04,0.02,0.21,0.15,0.13,0.11,0.23,0.21,0.19,0.15,0.20,0.21,0.21,0.22,0.31,0.55,0.31,0.17,0.37,0.54,0.31,0.17,0.87,0.56,0.40,0.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,M,"(3, 8)",2,4,101,0,2021-01
1,10101,2,2021,1,1,0.55,0.39,0.20,0.10,0.29,0.22,0.12,0.07,0.36,0.35,0.28,0.24,0.42,0.50,0.50,0.50,0.31,0.27,0.28,0.28,0.31,0.31,0.61,0.33,0.37,0.35,0.61,0.33,0.84,0.80,0.58,0.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,M,"(3, 8)",2,4,101,1,2021-02
2,10101,3,2021,1,1,1.00,0.79,0.41,0.21,0.38,0.41,0.23,0.12,0.38,0.47,0.39,0.33,0.24,0.37,0.36,0.33,0.20,0.24,0.24,0.25,0.31,0.31,0.95,0.51,0.37,0.35,0.95,0.51,0.93,0.99,0.71,0.53,1.00,1.00,1.00,1.00,1.00,1.00,0.68,0.60,1.00,1.00,1.00,1.00,M,"(3, 8)",2,4,101,1,2021-03
3,10101,4,2021,2,1,0.55,0.21,0.52,0.26,0.18,0.09,0.28,0.15,0.23,0.17,0.41,0.35,0.52,0.49,0.38,0.36,0.23,0.24,0.25,0.25,0.31,0.52,0.31,0.67,0.37,0.52,0.36,0.68,0.88,0.56,0.81,0.60,1.00,1.00,0.39,0.72,1.00,1.00,0.68,0.51,1.00,1.00,1.00,1.00,M,"(3, 8)",2,4,101,1,2021-04
4,10101,5,2021,2,1,0.45,0.39,0.61,0.31,0.20,0.19,0.34,0.18,0.23,0.26,0.46,0.39,0.60,0.52,0.41,0.39,0.23,0.24,0.25,0.25,0.31,0.31,0.31,0.85,0.37,0.35,0.36,0.87,0.94,0.88,0.90,0.66,1.00,1.00,0.39,0.72,1.00,1.00,0.68,0.51,0.00,1.00,1.00,1.00,M,"(3, 8)",2,4,101,1,2021-05
5,10101,6,2021,2,1,0.73,0.68,0.76,0.38,0.18,0.28,0.39,0.21,0.29,0.32,0.52,0.44,0.36,0.45,0.40,0.38,0.14,0.20,0.23,0.23,0.31,0.31,0.31,0.31,0.37,0.35,0.36,0.35,1.00,0.29,0.98,0.72,1.00,1.00,0.39,0.72,1.00,1.00,0.68,0.51,1.00,1.00,1.00,1.00,M,"(3, 8)",2,4,101,1,2021-06
6,10101,7,2021,3,2,0.27,0.11,0.06,0.41,0.13,0.06,0.04,0.23,0.21,0.15,0.13,0.44,1.00,0.95,0.97,0.43,0.32,0.33,0.34,0.24,0.31,0.63,0.36,0.31,0.37,0.67,0.39,0.35,0.29,0.66,0.47,0.78,1.00,1.00,0.50,0.72,1.00,1.00,1.00,0.51,0.00,0.00,0.00,1.00,M,"(3, 8)",2,4,101,1,2021-07
7,10101,8,2021,3,2,0.55,0.32,0.17,0.47,0.20,0.16,0.09,0.26,0.18,0.22,0.18,0.46,0.44,0.67,0.68,0.45,0.20,0.25,0.25,0.24,0.31,0.31,0.72,0.31,0.37,0.35,0.77,0.35,0.29,0.99,0.71,0.82,1.00,1.00,1.00,0.72,1.00,1.00,0.68,0.51,0.00,1.00,1.00,1.00,M,"(3, 8)",2,4,101,1,2021-08
8,10101,9,2021,3,2,0.45,0.50,0.26,0.51,0.17,0.25,0.14,0.28,0.23,0.29,0.24,0.47,0.45,0.55,0.56,0.44,0.27,0.26,0.26,0.24,0.31,0.31,0.31,0.31,0.37,0.35,0.36,0.35,0.98,0.29,0.85,0.85,1.00,1.00,0.39,0.72,1.00,1.00,0.68,0.51,0.00,1.00,1.00,

In [8]:
df_pipeline.drop(columns=['CUST_NUM_UNIQUE_SUBCAT_MONTH', 'CUST_NUM_UNIQUE_SUBCAT_QUARTER','CUST_TOTAL_QTY_BOUGHT_SEMESTER',
                'CUST_AVG_BASKET_SIZE_MONTH', 'CUST_AVG_BASKET_SIZE_QUARTER', 'CUST_AVG_BASKET_SIZE_YEAR','SUBCAT_NUM_TRANSACTIONS_MONTH',
                'SUBCAT_NUM_TRANSACTIONS_QUARTER','SUBCAT_NUM_TRANSACTIONS_SEMESTER','SUBCAT_TOTAL_QTY_BOUGHT_MONTH','SUBCAT_TOTAL_QTY_BOUGHT_SEMESTER',
                'SUBCAT_TOTAL_QTY_BOUGHT_YEAR','SUBCAT_NUM_UNIQUE_CUST_QUARTER','SUBCAT_NUM_UNIQUE_CUST_SEMESTER',
                 'SUBCAT_NUM_UNIQUE_CUST_YEAR','CUSTSUBCAT_NUM_TRANSACTIONS_MONTH','CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER',
                'CUSTSUBCAT_NUM_TRANSACTIONS_YEAR','CUSTSUBCAT_TOTAL_QTY_BOUGHT_MONTH','CUSTSUBCAT_TOTAL_QTY_BOUGHT_QUARTER',
                'CUSTSUBCAT_TOTAL_QTY_BOUGHT_YEAR'], inplace=True)

In [9]:
df_pipeline = df_pipeline.drop(columns=['QUARTER','SEMESTER','SEG_LIFESTAGE_CD','SEG_LIFESTYLE_CD','MONTH', 'SUBCAT_CD_EXT'])

In [10]:
# One-hot encoding categorical variables
df_pipeline = pd.get_dummies(df_pipeline, columns=['GENDER','FAMILY_MEMBERS','CAT_CD_EXT','YEAR'])

# Predictions for november 2022

In [11]:
RF = joblib.load('RF.joblib')

In [12]:
df_nov = df_pipeline[df_pipeline['FULLDATE']=='2022-11'].drop(columns=['FULLDATE'])

X_test = df_nov.drop(columns=['TARGET'])
y_test = df_nov['TARGET']

In [13]:
y_pred = RF.predict(X_test)

In [14]:
y_test = y_test.astype(int)
y_pred = y_pred.astype(int)

In [15]:
precision = metrics.precision_score(y_test, y_pred)
accuracy = metrics.accuracy_score(y_test, y_pred)
f1 = metrics.f1_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
roc_auc = metrics.auc(fpr, tpr)

print('Accuracy:  {:2.2%} '.format(accuracy))
print('Precision: {:2.2%} '.format(precision))
print('Recall:    {:2.2%} '.format(recall))
print('F1 Score:  {:2.2%} '.format(f1))
print('ROC-AUC:   {:2.2%} '.format(roc_auc))

Accuracy:  75.89% 
Precision: 33.62% 
Recall:    79.85% 
F1 Score:  47.32% 
ROC-AUC:   77.56% 


# Baseline model

In [16]:
df_out = df_pipeline[df_pipeline['FULLDATE']=='2022-09'].drop(columns=['FULLDATE'])
y_pred_baseline_nov = df_out[['TARGET']]

In [17]:
y_pred_baseline_nov = np.array(y_pred_baseline_nov).flatten().astype(int)

In [18]:
precision = metrics.precision_score(y_test, y_pred_baseline_nov)
accuracy = metrics.accuracy_score(y_test, y_pred_baseline_nov)
f1 = metrics.f1_score(y_test, y_pred_baseline_nov)
recall = metrics.recall_score(y_test, y_pred_baseline_nov)
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_baseline_nov, pos_label=1)
roc_auc = metrics.auc(fpr, tpr)

print('Accuracy:  {:2.2%} '.format(accuracy))
print('Precision: {:2.2%} '.format(precision))
print('Recall:    {:2.2%} '.format(recall))
print('F1 Score:  {:2.2%} '.format(f1))
print('ROC-AUC:   {:2.2%} '.format(roc_auc))

Accuracy:  85.43% 
Precision: 45.99% 
Recall:    42.90% 
F1 Score:  44.39% 
ROC-AUC:   67.50% 


In [19]:
test = pd.DataFrame(y_test)
test['PREDICTION_Baseline'] = y_pred_baseline_nov
test['PREDICTION_RandomForest'] = y_pred

In [20]:
test_df = pd.concat([test,df[['CUSTOMER_ACCOUNT_NR_MASK','GENDER', 'FAMILY_MEMBERS', 'SEG_LIFESTYLE_CD', 'SEG_LIFESTAGE_CD']]], axis=1).dropna()

In [21]:
test_df

,TARGET,PREDICTION_Baseline,PREDICTION_RandomForest,CUSTOMER_ACCOUNT_NR_MASK,GENDER,FAMILY_MEMBERS,SEG_LIFESTYLE_CD,SEG_LIFESTAGE_CD
22,0.00,1.00,1.00,37912,M,"(3, 8)",2,4
46,0.00,1.00,1.00,37912,M,"(3, 8)",2,4
70,0.00,0.00,0.00,37912,M,"(3, 8)",2,4
94,0.00,0.00,0.00,37912,M,"(3, 8)",2,4
118,0.00,0.00,0.00,37912,M,"(3, 8)",2,4
...,...,...,...,...,...,...,...,...
3365290,0.00,0.00,0.00,130366108,M,"(3, 8)",1,4
3365314,0.00,0.00,0.00,130366108,M,"(3, 8)",1,4
3365338,0.00,0.00,0.00,130366108,M,"(3, 8)",1,4
3365362,0.00,0.00,0.00,130366108,M,"(3, 8)",1,4


# Preparing data for RStudio

In [47]:
def R_dataframe(df, methods, metric):
    df_R = pd.DataFrame(columns=['CUSTOMER_ACCOUNT_NR_MASK'])
    customers = df['CUSTOMER_ACCOUNT_NR_MASK'].unique()

    for customer_id in customers:
        customer_data = df[df['CUSTOMER_ACCOUNT_NR_MASK'] == customer_id]
        y_test = customer_data['TARGET']

        row_data = {'CUSTOMER_ACCOUNT_NR_MASK': customer_id}

        for method in methods:
            method_col = f"{method}"
            fpr, tpr, thresholds = roc_curve(y_test, customer_data['PREDICTION' + '_' + method],  pos_label=1)
            metric_value = auc(fpr, tpr)
            row_data[method_col] = metric_value

        df_R = df_R.append(row_data, ignore_index=True)

    return df_R

df_R = R_dataframe(test_df, ['Baseline', 'RandomForest'], 'f1_score')

In [48]:
df_R

,CUSTOMER_ACCOUNT_NR_MASK,Baseline,RandomForest
0,37912.00,0.66,0.81
1,177299.00,0.75,0.74
2,179359.00,0.67,0.69
3,202298.00,0.58,0.71
4,295501.00,0.48,0.67
...,...,...,...
995,120567137.00,0.71,0.81
996,125228790.00,0.70,0.91
997,128398028.00,NaN,NaN
998,129693269.00,0.54,0.93


In [49]:
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
customer_dict = dict(zip(df['CUSTOMER_ACCOUNT_NR_MASK'], df['GENDER']))
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
family_dict = dict(zip(df['CUSTOMER_ACCOUNT_NR_MASK'], df['FAMILY_MEMBERS']))
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
lifestyle_dict = dict(zip(df['CUSTOMER_ACCOUNT_NR_MASK'], df['SEG_LIFESTYLE_CD']))
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
lifestage_dict = dict(zip(df['CUSTOMER_ACCOUNT_NR_MASK'], df['SEG_LIFESTAGE_CD']))


# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
df_R['GENDER'] = df_R['CUSTOMER_ACCOUNT_NR_MASK'].map(customer_dict)
# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
df_R['FAMILY_MEMBERS'] = df_R['CUSTOMER_ACCOUNT_NR_MASK'].map(family_dict)
# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
df_R['SEG_LIFESTYLE_CD'] = df_R['CUSTOMER_ACCOUNT_NR_MASK'].map(lifestyle_dict)
# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
df_R['SEG_LIFESTAGE_CD'] = df_R['CUSTOMER_ACCOUNT_NR_MASK'].map(lifestage_dict)

In [50]:
df_R

,CUSTOMER_ACCOUNT_NR_MASK,Baseline,RandomForest,GENDER,FAMILY_MEMBERS,SEG_LIFESTYLE_CD,SEG_LIFESTAGE_CD
0,37912.00,0.66,0.81,M,"(3, 8)",2,4
1,177299.00,0.75,0.74,F,"(3, 8)",1,3
2,179359.00,0.67,0.69,M,"(3, 8)",2,4
3,202298.00,0.58,0.71,F,"(3, 8)",2,3
4,295501.00,0.48,0.67,F,"(3, 8)",2,4
...,...,...,...,...,...,...,...
995,120567137.00,0.71,0.81,M,"(1, 2)",2,3
996,125228790.00,0.70,0.91,M,"(3, 8)",1,5
997,128398028.00,NaN,NaN,F,"(1, 2)",3,4
998,129693269.00,0.54,0.93,F,"(3, 8)",2,5


In [51]:
df_R['SEG_LIFESTAGE_CD'].value_counts()

5    441
4    426
3    103
2     20
6      9
1      1
Name: SEG_LIFESTAGE_CD, dtype: int64

In [52]:
df_R_balanced = df_R[(df_R['SEG_LIFESTAGE_CD'] == 5) | (df_R['SEG_LIFESTAGE_CD'] == 4)]

In [53]:
df_R_balanced = df_R_balanced[(df_R_balanced['FAMILY_MEMBERS'] == '(3, 8)') | (df_R_balanced['FAMILY_MEMBERS'] == '(1, 2)')]

In [54]:
df_R_balanced = df_R_balanced.fillna(0)

In [55]:
df_R_balanced[df_R_balanced['CUSTOMER_ACCOUNT_NR_MASK']==295501]

,CUSTOMER_ACCOUNT_NR_MASK,Baseline,RandomForest,GENDER,FAMILY_MEMBERS,SEG_LIFESTYLE_CD,SEG_LIFESTAGE_CD
4,295501.00,0.48,0.67,F,"(3, 8)",2,4


In [56]:
df_R_balanced

,CUSTOMER_ACCOUNT_NR_MASK,Baseline,RandomForest,GENDER,FAMILY_MEMBERS,SEG_LIFESTYLE_CD,SEG_LIFESTAGE_CD
0,37912.00,0.66,0.81,M,"(3, 8)",2,4
2,179359.00,0.67,0.69,M,"(3, 8)",2,4
4,295501.00,0.48,0.67,F,"(3, 8)",2,4
6,396634.00,0.50,0.77,F,"(3, 8)",2,4
7,423754.00,0.74,0.81,F,"(3, 8)",1,5
...,...,...,...,...,...,...,...
994,112467482.00,0.70,0.67,M,"(1, 2)",1,4
996,125228790.00,0.70,0.91,M,"(3, 8)",1,5
997,128398028.00,0.00,0.00,F,"(1, 2)",3,4
998,129693269.00,0.54,0.93,F,"(3, 8)",2,5


In [57]:
df_R_balanced.to_csv('rocauc_SONAE.csv', index=False)